# Lesson [YOUR_NR]: [LESSON_TITLE]

This notebook explores **X** for Y.

We will use the `google-genai` library to interact with Google's Gemini models.

**Learning Objectives:**

1.  

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:

In [1]:
%load_ext autoreload
%autoreload 2

### Set Up Python Environment

To set up your Python virtual environment using `uv` and load it into the Notebook, follow the step-by-step instructions from the `Course Admin` lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.

### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions from the `Course Admin` lesson.

But here is a quick check on what you need to run this Notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  From the root of your project, run: `cp .env.example .env` 
3.  Within the `.env` file, fill in the `GOOGLE_API_KEY` variable:

Now, the code below will load the key from the `.env` file:

In [ ]:
from lessons.utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

### Import Key Packages

In [ ]:
from google import genai
from google.genai import types

from lessons.utils import pretty_print

### Initialize the Gemini Client

In [4]:
client = genai.Client()

### Define Constants

We will use the `gemini-2.5-flash` model, which is fast and cost-effective:

In [ ]:
MODEL_ID = "gemini-2.5-flash"

## 2. [YOUR_IMPLEMENTATION]


